筆記內容置於我的Notion帳戶中，網址如下：  
網路爬蟲：https://www.notion.so/218034b941d14ac08c1032c7f0ec88b1  

程式碼如下：

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
ur1_home = 'https://www.books.com.tw/web/books_nbtopm_01/?o=5&v=1'
url = "https://www.books.com.tw/web/books_nbtopm_"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}
html = requests.get(ur1_home, headers=headers) #headers：加上標頭，偽裝成一般使用者，而非爬蟲，避免該網站擋爬蟲
html.encoding = 'utf-8'
soup = BeautifulSoup(html.text, 'lxml')

In [ ]:
res = soup.find('div', class_='mod_b type02_l001-1 clearfix') #因為class是python的關鍵字，所以加底線
hrefs = res.select("a") 
print(hrefs)

[<a href="https://www.books.com.tw/web/books_nbtopm_01/?loc=P_0003_001">文學小說</a>, <a href="https://www.books.com.tw/web/books_nbtopm_02/?loc=P_0003_002">商業理財</a>, <a href="https://www.books.com.tw/web/books_nbtopm_03/?loc=P_0003_003">藝術設計</a>, <a href="https://www.books.com.tw/web/books_nbtopm_04/?loc=P_0003_004">人文史地</a>, <a href="https://www.books.com.tw/web/books_nbtopm_05/?loc=P_0003_005">社會科學</a>, <a href="https://www.books.com.tw/web/books_nbtopm_07/?loc=P_0003_006">心理勵志</a>, <a href="https://www.books.com.tw/web/books_nbtopm_12/?loc=P_0003_007">宗教命理</a>, <a href="https://www.books.com.tw/web/books_nbtopm_06/?loc=P_0003_008">自然科普</a>, <a href="https://www.books.com.tw/web/books_nbtopm_08/?loc=P_0003_009">醫療保健</a>, <a href="https://www.books.com.tw/web/books_nbtopm_09/?loc=P_0003_010">飲食</a>, <a href="https://www.books.com.tw/web/books_nbtopm_10/?loc=P_0003_011">生活風格</a>, <a href="https://www.books.com.tw/web/books_nbtopm_11/?loc=P_0003_012">旅遊</a>, <a href="https://www.books.com.

In [ ]:
category_no = 1
mode = '/?o=5&v=1' #顯示模式為直式；排序依暢銷度

In [ ]:
#將數字轉為2個數字的字串
def twobyte(category_no):
  if category_no < 10:
    num_twostr = '0' + str(category_no)
  else:
    num_twostr = str(category_no)
  return num_twostr

In [ ]:
print(twobyte(1))

01


In [ ]:
#顯示該分類書籍總頁數，以及各頁數的網址
def showpages(url, category):
  html = requests.get(url, headers=headers).text
  soup = BeautifulSoup(html, 'lxml')
  try:
    #總頁數
    pages = int(soup.select('.page span')[0].text) #取class=page中的第1個span
    print("共有", pages, "頁")
    
    for page in range(1, pages+1):
      page_url = url + '&page=' + str(page)
      print("第", page, "頁", page_url)
  except : #沒有分頁的處理
    print("共有", 1, "頁")

In [ ]:
showpages('https://www.books.com.tw/web/books_nbtopm_02/?o=5&v=1', '商業理財')

共有 3 頁
第 1 頁 https://www.books.com.tw/web/books_nbtopm_02/?o=5&v=1&page=1
第 2 頁 https://www.books.com.tw/web/books_nbtopm_02/?o=5&v=1&page=2
第 3 頁 https://www.books.com.tw/web/books_nbtopm_02/?o=5&v=1&page=3


In [ ]:
#儲存該分類所有書籍的資料於list中
def getbook(url, category):

  html = requests.get(url, headers=headers).text
  soup = BeautifulSoup(html, 'lxml')

  #總頁數
  pages = int(soup.select('.page span')[0].text)
  books_list=[]

  for page in range(1, pages+1):

    page_url = url + '&page=' + str(page)
    html = requests.get(page_url, headers=headers).text
    soup = BeautifulSoup(html, 'lxml')

    res = soup.find('div', {'class':'mod type02_m012 clearfix'})
    items = res.select('.item') #所有item
    for item in items:

      This_list=[]
      msg = item.select('.msg')[0]

      title = msg.select('a')[0].text
      author = msg.select('a')[1].text
      publish = msg.select('a')[2].text
      publish_date = msg.find('span').text.split("：")[-1]
      discount = item.select('.price .set2')[0].text
      content = item.select('.txt_cont')[0].text.replace(" ", '')
      This_list = [category, title, author, publish, publish_date, discount, content]
      books_list.append(This_list)

  return books_list

In [ ]:
import requests
from bs4 import BeautifulSoup
import openpyxl
from time import sleep

category_no = int(input("請輸入要下載的分類:"))
mode = '/?o=5&v=1' #顯示模式為直式；排序依暢銷度

if 0 < category_no <= len(hrefs):

  category =  hrefs[category_no-1].text #分類名稱
  print("下載的分類編號:{} 分類名稱:{}".format(category_no, category))

  #該分類網址
  category_ur1 = url + twobyte(category_no) + mode #twobyte()：將數字轉為2個數字的字串
  #e.g. 'https://www.books.com.tw/web/books_nbtopm_02/?o=5&v=1'
  print('分類的網址：' + category_ur1)

  #顯示該分類書籍總頁數，以及各頁數的網址
  showpages(category_ur1, category)
  
  print("資料寫入Exce1中，請等候一會!")
  workbook = openpyxl.Workbook()
  sheet = workbook.worksheets[0]
  title = ["分類","書名","作者","出版社","出版日期","優惠價","內容"]
  sheet.append(title)
  books_list = getbook(category_ur1, category)
  for item in books_list: 
    sheet.append(item) 
    sleep(0.1) #加上delay時間
  workbook.save('books.xlsx')
  print("已完成儲存！")
else:
  print('分類不存在！')

請輸入要下載的分類:2
下載的分類編號:2 分類名稱:商業理財
分類的網址：https://www.books.com.tw/web/books_nbtopm_02/?o=5&v=1
共有 3 頁
第 1 頁 https://www.books.com.tw/web/books_nbtopm_02/?o=5&v=1&page=1
第 2 頁 https://www.books.com.tw/web/books_nbtopm_02/?o=5&v=1&page=2
第 3 頁 https://www.books.com.tw/web/books_nbtopm_02/?o=5&v=1&page=3
資料寫入Exce1中，請等候一會!
已完成儲存！


In [ ]:
#顯示該分類所有書籍
def showbooks(url, category):
  
  html = requests.get(url, headers=headers).text
  soup = BeautifulSoup(html, 'lxml')

  #總頁數
  pages = int(soup.select('.page span')[0].text)

  for page in range(1, pages+1):

    page_url = url + '&page=' + str(page)
    html = requests.get(page_url, headers=headers).text
    soup = BeautifulSoup(html, 'lxml')

    res = soup.find('div', {'class':'mod type02_m012 clearfix'})
    items = res.select('.item') #所有item
    for item in items:
      msg = item.select('.msg')[0]

      title = msg.select('a')[0].text
      print('書名:', title)
      author = msg.select('a')[1].text
      print('作者:', author)
      publish = msg.select('a')[2].text
      print('出版社:', publish)
      publish_date = msg.find('span').text.split("：")[-1]
      print('出版日期:', publish_date)
      discount = item.select('.price .set2')[0].text
      print(discount)
      content = item.select('.txt_cont')[0].text.replace(" ", '')
      print("內容:", content)
      print('---------------------------------------------------------') 

In [ ]:
showbooks('https://www.books.com.tw/web/books_nbtopm_02/?o=5&v=1', '商業理財')

書名: 變化中的世界秩序：橋水基金應對國家興衰的原則
作者: 瑞‧達利歐
出版社: 商業周刊
出版日期: 2022/04/28
優惠價：79折 514元
內容: 
★《原則》作者達利歐最新力作，唯一中文完整版！
★在變動世界裡找到不變的「大週期」，
定位現在、推測未來、預作準備
★美中爭霸、誰上誰下？橋水基金用演算法打分數！
算出衝突202...more

---------------------------------------------------------
書名: 卡片盒筆記：最高效思考筆記術，德國教授超強秘技，促進寫作、學習與思考，使你洞見源源不斷，成為專家
作者: 申克‧艾倫斯
出版社: 遠流
出版日期: 2022/04/27
優惠價：79折 315元
內容: 

正體中文版率先譯自原著2022年春季內容全面增訂版
收入作者獨家洞見〈魯曼卡片盒一瞥〉最新專文
解說如何將卡片盒使用在筆記軟體上。
★ ★ ★
本書告訴你howtotakesmartnotes.
聰明的筆記不是...more

---------------------------------------------------------
書名: 最美好、也最殘酷的翻身時代：畢德歐夫帶你掌握理財5大關鍵
作者: 畢德歐夫
出版社: 先覺
出版日期: 2022/05/01
優惠價：79折 277元
內容: 
無數富人從房市或股市中誕生，
既然如此，我們為什麼不學投資？
 
「金錢」「資產」「債務」「房產」「股市」
掌握這5大關鍵，平凡人也能創造出不凡的財富！
 
●如何用現有的100萬，擴增為更多...more

---------------------------------------------------------
書名: 隱市致富地圖：60億操盤手用一張圖，找到上漲超過30％的翻轉人生贏勢股
作者: 股市隱者
出版社: 大是文化
出版日期: 2022/03/29
優惠價：79折 332元
內容: 

臉書粉專粉絲數2年內突破4萬人，

ApplePodcast及Spotify上線一週即榮登排行榜第1名。

 

◎買股票像挖金礦，挖礦的工具很多，不必樣樣嫻熟，你要做的是挖對方向。

◎...more

------------------------------------